In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline

In [2]:
%run query_features.py
%run scripts/helper.py
%run scripts/model_train_plus_test.py

In [3]:
crowd_train = load_file('./data/train.csv/train.csv', None)
crowd_test = load_file('./data/test.csv/test.csv', None)

In [4]:
crowd_train = crowd_train[crowd_train.relevance_variance < 0.5]

In [5]:
target = crowd_train.median_relevance.values

In [49]:
# train_index, test_index = ssSplit(target, train_size=8000, random_state=44)
train_index, test_index = ssSplit(target, train_size=1000, random_state=44)

In [50]:
Xt = crowd_train.iloc[train_index]
Xv = crowd_train.iloc[test_index]

# Xt = crowd_train
# Xv = crowd_test

In [51]:
yt = target[train_index]
yv = target[test_index]

# yt = target

In [52]:
Xt_tweaked = tweak_text(Xt)
Xv_tweaked = tweak_text(Xv)

In [53]:
Xfitted, tfv = TFIDF(Xt_tweaked, None)

In [54]:
svd = TruncatedSVD(n_components=200, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)

scl = StandardScaler(copy=True, with_mean=True, with_std=True)

clf = SVC(C=10.0, kernel='linear', degree=3, 
        gamma=0.0, coef0=0.0, shrinking=True, probability=False, 
        tol=0.001, cache_size=200, class_weight=None, 
        verbose=False, max_iter=-1, random_state=None)

keywords = keyword_counter(Xt)

In [55]:
features = stack([keywords, Xfitted])

In [56]:
features_svd = svd.fit_transform(features)

In [57]:
features_scl = scl.fit_transform(features_svd)

### Feature selection

In [58]:
kappa_scorer = get_kappa_scorer()

In [ ]:
selector = feature_selection(features_scl, yt, clf, 1, None, kappa_scorer, None, 0)

In [26]:
selector.grid_scores_

array([ 0.09551443,  0.08587044,  0.09263525,  0.10222593,  0.10446155,
        0.11196421,  0.11980537,  0.1532042 ,  0.18316295,  0.18411899,
        0.19283421,  0.18717257,  0.23314848,  0.203963  ,  0.20825379,
        0.26279836,  0.30424894,  0.32203741,  0.30186082,  0.25278024,
        0.31742312,  0.32569698,  0.30012057,  0.27447408,  0.31833882,
        0.31342121,  0.31756955,  0.26548062,  0.22389084,  0.23368437,
        0.21912735,  0.21356475,  0.21658899,  0.18873339,  0.197842  ,
        0.21829272,  0.20929388,  0.21696621,  0.21999229,  0.24068457,
        0.23622419,  0.20397825,  0.21827953,  0.21135912,  0.23446343,
        0.19750452,  0.20252151,  0.18871767,  0.17076826,  0.10988198,
        0.13213942,  0.14247887,  0.17133736,  0.16121156,  0.13371489,
        0.16145429,  0.16127367,  0.17602269,  0.17288821,  0.15778934,
        0.17682649,  0.15262014,  0.17968618,  0.17732637,  0.17032855,
        0.17693857,  0.16577611,  0.1756472 ,  0.14748799,  0.13

In [27]:
best_estimator = selector.estimator_

In [36]:
selector.

22

### Predict on test set

In [28]:
keywords_test = keyword_counter(Xv)

In [29]:
Xtest_fitted  = tfv.transform(Xv_tweaked)

In [30]:
features_test = stack([keywords_test, Xtest_fitted])

In [32]:
features_svd_test = svd.transform(features_test)

In [33]:
features_scl_test = scl.transform(features_svd_test)

In [37]:
features_selected_test = selector.transform(features_scl_test)

In [39]:
print 'Weighted kappa score on test set %0.4f ' % quadratic_weighted_kappa(yv, best_estimator.predict(features_selected_test))

Weighted kappa score on test set 0.2867 
